In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [37]:
## load the ANN trained model 

model = load_model('model.h5')

## load scaler pickle file , onehot encode pickle file

with open('onehot_encoder_geography.pkl','rb') as file:
    label_encoder_geography=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [38]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [39]:
## exmaple input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Female',
    'Age':40,
    'Tenure':3,
    'Balance':600000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [40]:
geo_encoded = label_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geography.get_feature_names_out(['Geography']))

c:\Users\anees\miniconda3\envs\tf_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [41]:
input_df = pd.DataFrame([input_data])

In [42]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [43]:
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,40,3,600000,2,1,1,50000


In [44]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [45]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,600000,2,1,1,50000


In [46]:
## concatenation one hot encoded
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [47]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,600000,2,1,1,50000,1.0,0.0,0.0


In [48]:
## scaling the input data
input_scaled = scaler.transform(input_df)

In [49]:
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349,  8.38812313,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [50]:
## prediction
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step


In [51]:
print(prediction[0][0])

0.8888376


In [52]:
if prediction > 0.5:
    print(f"Customer is likely to churn :{prediction[0][0]}")
else:
    print(f"Customer is not likely to churn:{prediction[0][0]}")

Customer is likely to churn :0.8888375759124756
